In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import re

In [3]:
def IR_Flag(rate):
    if rate > .12:
        return True
    else:
        return False

In [4]:
def num_strip_rt(data):
    if '%' in data:
        data = data.rstrip('%')
        
    if "." in data:
        data = float(data)/100
        
    return data
    

In [5]:
def get_int_from_text(data):
#     data = re.search('[0-9]*', data).group(0)
    data = re.findall('[0-9]+', data) # produces a 1 item list
    
#     print("data is type {}".format(type(data)))
    
#     data = [x for x in data if x != '']

    try:
        return data[0]
    except: 
        return -1
    
    
def own(data):
    if data in ['MORTGAGE','OWN']:
        return 1
    else:
        return 0
                     

In [6]:
# get loandata
raw_data = pd.read_csv('https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv')

In [7]:
raw_data.dropna(inplace=True) # drop the na rows
# clean fico range and get score 
raw_data['FICO.Score'] = map(lambda x: int(x.split('-')[0]), raw_data['FICO.Range']) # clean fico range and get score #ask Kyle why this runs so slow

raw_data['Debt.To.Income.Ratio'] = [num_strip_rt(data) for index, data in raw_data['Debt.To.Income.Ratio'].iteritems()]
raw_data['Debt.To.Income.Ratio'].astype(float)
raw_data['Interest.Rate'] = [num_strip_rt(data) for index, data in raw_data['Interest.Rate'].iteritems()]
raw_data['Loan.Length'] = [get_int_from_text(data) for index, data in raw_data['Loan.Length'].iteritems()]
raw_data['Employment.Length2'] = [get_int_from_text(data) for index, data in raw_data['Employment.Length'].iteritems()]
raw_data = raw_data[raw_data['Employment.Length2'] > 0] # delete any employment length -1 generated by original N/A
raw_data['Loan.Length'] = raw_data['Loan.Length'].astype(int)
 # add rate flag
raw_data['IR_TF'] = [IR_Flag(rate) for index, rate in raw_data['Interest.Rate'].iteritems()]
raw_data['OWN'] = [own(desc) for index, desc in raw_data['Home.Ownership'].iteritems()]
raw_data['constant']= 1

# car                     50
# credit_card            431
# debt_consolidation    1267
# educational             14
# home_improvement       147
# house                   20
# major_purchase         100
# medical                 28
# moving                  29
# other                  189
# renewable_energy         4
# small_business          84
# vacation                20
# wedding                 38





In [8]:
gbhome = raw_data.groupby(['Loan.Purpose'])

In [9]:
gbhome.size()

Loan.Purpose
car                     50
credit_card            431
debt_consolidation    1267
educational             14
home_improvement       147
house                   20
major_purchase         100
medical                 28
moving                  29
other                  189
renewable_energy         4
small_business          84
vacation                20
wedding                 38
dtype: int64

In [10]:
tmp_ind_vars = list(raw_data.columns.values)

In [11]:
tmp_ind_vars

['Amount.Requested',
 'Amount.Funded.By.Investors',
 'Interest.Rate',
 'Loan.Length',
 'Loan.Purpose',
 'Debt.To.Income.Ratio',
 'State',
 'Home.Ownership',
 'Monthly.Income',
 'FICO.Range',
 'Open.CREDIT.Lines',
 'Revolving.CREDIT.Balance',
 'Inquiries.in.the.Last.6.Months',
 'Employment.Length',
 'FICO.Score',
 'Employment.Length2',
 'IR_TF',
 'OWN',
 'constant']

In [12]:
ind_vars = ['Amount.Requested',
 'Amount.Funded.By.Investors','Interest.Rate','Loan.Length','Monthly.Income',
    'Open.CREDIT.Lines','Revolving.CREDIT.Balance','Inquiries.in.the.Last.6.Months',
    'FICO.Score','OWN','constant']
#             ,
#  'Employment.Length2', # problems with this field

# ,'Debt.To.Income.Ratio'] # problems with this field

 

In [13]:
ind_vars2 = ['FICO.Score','Amount.Funded.By.Investors','constant']

In [14]:
logit = sm.Logit(raw_data['IR_TF'], raw_data[ind_vars2])

In [15]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.323165
         Iterations 8


In [16]:
coeff = result.params

In [17]:
print(coeff)

FICO.Score                    -0.085928
Amount.Funded.By.Investors     0.000169
constant                      59.200174
dtype: float64


In [18]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  IR_TF   No. Observations:                 2421
Model:                          Logit   Df Residuals:                     2418
Method:                           MLE   Df Model:                            2
Date:                Thu, 25 Feb 2016   Pseudo R-squ.:                  0.5183
Time:                        09:20:28   Log-Likelihood:                -782.38
converged:                       True   LL-Null:                       -1624.2
                                        LLR p-value:                     0.000
==============================================================================================
                                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
FICO.Score                    -0.0859      0.004    -24.398      0.000        -0.093    -0.079
Amount.Funded.By.Investors     0.0002   1.11e-05     15.170      0.000         0.000     0.000
constant                      59.2002      2.420     24.462      0.000        54.457    63.943
==============================================================================================
"""

In [20]:
result.predict([750,10000])

ValueError: shapes (1,2) and (3,) not aligned: 2 (dim 1) != 3 (dim 0)

In [196]:
result.predict({'FICO.Score':750,'Amount.Funded.By.Investors': 10000,'constant':1}) # pass as dict

ValueError: shapes (1,1) and (3,) not aligned: 1 (dim 1) != 3 (dim 0)